In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark: SparkSession = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

df.show()

Py4JJavaError: An error occurred while calling o25.addFile.
: java.lang.RuntimeException: java.io.FileNotFoundException: Hadoop bin directory does not exist: C:\Users\spark-3.5.1-bin-hadoop3\bin\bin -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1139)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1125)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:489)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1795)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1733)
	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:103)
	at java.base/java.lang.reflect.Method.invoke(Method.java:580)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(Reflecti

In [ ]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('Sales')

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
          
          select round(avg(price), 2)

          from Sales 

          where bedrooms = 4

""").show()


+--------------------+
|round(avg(price), 2)|
+--------------------+
|           299661.01|
+--------------------+



In [4]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
          
          select date_built, round(avg(price), 2)

          from Sales

          where bedrooms = 3 and bathrooms = 3 
          
          group by date_built 
          
          order by date_built
          
""").show()


AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `Sales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 5 pos 15;
'Sort ['date_built ASC NULLS FIRST], true
+- 'Aggregate ['date_built], ['date_built, unresolvedalias('round('avg('price), 2), None)]
   +- 'Filter (('bedrooms = 3) AND ('bathrooms = 3))
      +- 'UnresolvedRelation [Sales], [], false


In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
          
          select date_built, round(avg(price), 2)
          
          from Sales 
          
          where bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >= 2000
          
          group by date_built 
          
          order by date_built

""").show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           285010.22|
|      2011|           276553.81|
|      2012|           307539.97|
|      2013|           303676.79|
|      2014|           298264.72|
|      2015|           297609.97|
|      2016|            293965.1|
|      2017|           280317.58|
+----------+--------------------+



In [5]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
          
          select view, round(avg(price), 2)
          
          from Sales 
          
          group by view 
          
          having avg(price) >= 350000
          
          order by view desc

""").show()

print("--- %s seconds ---" % (time.time() - start_time))

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `Sales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 5 pos 15;
'Sort ['view DESC NULLS LAST], true
+- 'UnresolvedHaving ('avg('price) >= 350000)
   +- 'Aggregate ['view], ['view, unresolvedalias('round('avg('price), 2), None)]
      +- 'UnresolvedRelation [Sales], [], false


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("Cache table Sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('Sales')

True

In [11]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
          
          select view, round(avg(price), 2)
          
          from Sales 
          
          group by view 
          
          having avg(price) >= 350000
          
          order by view desc

          """).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
| 100|           1026669.5|
|  99|          1061201.42|
|  98|          1053739.33|
|  97|          1129040.15|
|  96|          1017815.92|
|  95|           1054325.6|
|  94|           1033536.2|
|  93|          1026006.06|
|  92|           970402.55|
|  91|          1137372.73|
|  90|          1062654.16|
|  89|          1107839.15|
|  88|          1031719.35|
|  87|           1072285.2|
|  86|          1070444.25|
|  85|          1056336.74|
|  84|          1117233.13|
|  83|          1033965.93|
|  82|           1063498.0|
|  81|          1053472.79|
+----+--------------------+
only showing top 20 rows

--- 0.319927453994751 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").parquet("Sales_Partitioned", mode = 'overwrite')

Py4JJavaError: An error occurred while calling o62.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(Fil

In [ ]:
# 11. Read the formatted parquet data.
p_sales_df_p = spark.read.parquet('Sales_Partitioned')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_sales_df_p.createOrReplaceTempView('p_sales_p')

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
          
          select view, round(avg(price), 2) as Average_Price 
          
          from p_sales_p 
          
          group by view 
          
          having avg(price) >=350000
          
          order by view desc

          """).show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table sales")

In [ ]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('sales')